# Part 3 - Interactivity in Vega-Altair

- **Objective**: Learn how to add interactivity to charts.
- **Key Topics**:
  - Basic Interactive features (tooltips and pan/zoom)
  - Selections parameters
  - Conditional encodings and filtering

One of the most exciting features of Vega-Altair is its ability to produce interactive charts. These interactive visualizations can enhance the user's data exploration experience by allowing dynamic manipulation and immediate feedback without the need for continuous Python interaction. This means that once created, these interactive charts are fully functional in a web browser without the need for a Python kernel.

In this section, we'll explore the various interactive features offered by Vega-Altair, ranging from simple tooltips to complex, multi-view charts. To conclude, we'll discuss Vega-Altair's Jupyter Widget integration that makes it possible to interact with the interactive features of a chart from Python.


## Imports

In [ ]:
import pandas as pd
import altair as alt
from vega_datasets import data
import vegafusion

print("The installed Vega-Altair version is " + alt.__version__)

## Introduction to Interactive Features in Vega-Altair

Vega-Altair allows you to create interactive charts that can be embedded in web pages or Jupyter notebooks (without requiring a live Python kernel). The interactivity features include tooltips, panning, zooming, selections, and more complex interactions such as linked views and coordinated highlighting. These features enable dynamic data exploration and enhance the end user experience by making visualizations more engaging and insightful.

Compared to static charts, interactive charts provide a more personalized experience, allowing end users to ask their own questions and focus on their particular areas of interest.

### Dataset

The cars dataset from vega_datasets contains information about various car models from the 1970s and 1980s. 
It includes properties like the horsepower, miles per gallon, origin.


In [ ]:
cars = data.cars()
cars.head()

### Example: Support pan, zoom, and tooltips

In [ ]:
# Create chart with tooltip
alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon').title("Miles per Gallon"),
    alt.Color('Origin'),
    tooltip=[
        'Name',
        'Origin',
        'Horsepower',
        alt.Tooltip('Miles_per_Gallon').title("Miles per Gallon")
    ]
).interactive()

### Example: Support end-user filtering

In [ ]:
options = ['Europe', 'Japan', 'USA']
labels = [option + ' ' for option in options]

input_radio = alt.binding_radio(
    options=options + [None],
    labels=labels + ['All'],
    name='Region: '
)
selection = alt.selection_point(
    fields=['Origin'],
    value=None,
    bind=input_radio,
)

alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower:Q'),
    alt.Y('Miles_per_Gallon:Q'),
    alt.Color('Origin:N').scale(domain=options),
).add_params(
    selection
).transform_filter(
    selection
)


### Example: Support multi-view highlighting

Interactive charts also enable the exploration higher dimensional relationships in the dataset through the use of selections across multi-view charts.

In [ ]:
brush = alt.selection_interval(encodings=['x'])

left_chart = alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(brush, 'Origin', alt.value('lightgray'))
).add_params(
    brush
)

right_chart = alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Acceleration'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(brush, 'Origin', alt.value('lightgray'))
).add_params(
    brush
)

left_chart | right_chart

## Basic Interactive Features

Vega-Altair provides high-level support for two basic forms of interactivity: Tooltips and Pan/Zoom interactions.

### Tooltips

As seen already in Part 1, tooltips provide additional information when a user hovers over a data point on the chart. This feature is particularly useful for displaying detailed data without cluttering the visualization.

Vega-Altair provides a `tooltip` encoding channel that may be used to both activate tooltip support, and specify which data values should appear in the tooltips.

In [ ]:
# Create chart with tooltip
alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    alt.Color('Origin'),
    tooltip=['Name', 'Origin', 'Horsepower', 'Miles_per_Gallon']
)

In [ ]:
# Create chart with tooltip
alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon').title("Miles per Gallon"),
    alt.Color('Origin'),
    tooltip=[
        'Name',
        'Origin',
        'Horsepower',
        alt.Tooltip('Miles_per_Gallon').title("Miles per Gallon")
    ]
)

### Pan and Zoom
Pan and zoom interactions allow users to navigate through different sections of the chart by dragging the view (panning) or changing the scale of the view (zooming). This interaction is useful for exploring large datasets where displaying all data points simultaneously would be impractical.

Pan and zoom interactions may be enabled on a Vega-Altair chart by calling the `chart.interactive()` method. Here's how to enable pan and zoom on a scatter plot above:

In [ ]:
# Create chart with tooltip
alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon').title("Miles per Gallon"),
    alt.Color('Origin'),
    tooltip=[
        'Name',
        'Origin',
        'Horsepower',
        alt.Tooltip('Miles_per_Gallon').title("Miles per Gallon")
    ]
).interactive()

> As we'll see later, `.interactive()` is a convenience method that enables pan and zoom on the chart using Vega-Altair's more general selections framework.

## Selection Parameters

A selection parameter represents a predicate expression (more on this below) that may be used to determine whether each row of a dataset is included in the selection or not.

Selection parameters may be bound to input widgets, or to interactions on the chart itself such as clicking on marks or clicking and dragging to create a box selection.

### What's a Predicate?
A predicate is a function or expression that evaluates to a boolean value (true or false). For example, in the context of the cars dataset we've been using, the expression `Origin == "Japan"` is a predicate that evaluates to true for cars with `Origin` of `Japan`, and false otherwise.  The exact predicate that a selection parameter represents is not always obvious, but viewing them as predicates is helpful in understanding how selections may be used.

### Conditions & Filters
Selection parameters can be used to influence the appearance of a chart in two ways: Filter and conditions.

#### Filter
The `.transform_filter()` method that we discussed in the previous section can accept a selection instance. In this case, the filter transform will remove all rows that don't satisfy the selection parameter's underlying predicate.

#### Condition
Selection parameters may also be used to control visual encoding channels using the `alt.condition()` function. A common scenario is the set the color or opacity of marks based on the selection, but the selection parameter may be used to control any encoding channel.

### Point selections
Vega-Altair supports two selection parameter types: *Point selections* and *interval selections*. Let's look at point selections first. A point selection represents a predicate of the form `{Column} == {value}`. In the case of the cars dataset, one such example would be `Origin == "Japan"`.  A point selection is constructed using the `alt.selection_point` function.

Here is a basic example of configuring a selection parameter with a fixed predicate (equivalent to `Origin == "Japan"`), and then using the selection to filter the chart's underlying data.

In [ ]:
selection = alt.selection_point(
    fields=['Origin'],
    value="Japan",
)

alt.Chart(cars).transform_filter(
    selection
).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.Color('Origin:N').scale(domain=['Europe', 'Japan', 'USA']),
).add_params(
    selection
)

Alternatively, the selection may be used to control the color of the selected points (without filtering) by passing a `condition` as the color encoding.

In [ ]:
selection = alt.selection_point(
    fields=['Origin'],
    value="Japan",
)

alt.Chart(cars).mark_point().encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(
        selection,
        alt.Color('Origin:N').scale(domain=['Europe', 'Japan', 'USA']),
        alt.value("lightgrey")
    )
).add_params(
    selection
)

> **Note:** Encoding channels that use a condition must be assigned to the corresponding keyword argument in the `.encode()` method call. For example, in this case that's `color=alt.condition(...)`.

### Point selection binding
While selection parameters may be configured statically as in the examples above, they are most powerful when bound to other components of the chart that may be manipulated interactively. Point selections may be bound to widgets, mark click/hover events, and to legend interactions.

#### Binding point selections to widgets
Point selections may be bound to input widgets. For example, let's extend the previous example to use a dropdown widget to control which country of origin is selected





In [ ]:
# construct dropdown (select) widget
input_dropdown = alt.binding_select(
    options=['Europe', 'Japan', 'USA'],
    name='Region '
)

selection = alt.selection_point(
    fields=['Origin'],
    value="Japan",
    bind=input_dropdown
)

alt.Chart(cars).mark_point().encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    order=alt.condition(selection, alt.value(1), alt.value(0)),
    color=alt.condition(
        selection,
        alt.Color('Origin:N').scale(domain=['Europe', 'Japan', 'USA']),
        alt.value("lightgrey")
    )
).add_params(
    selection
)

Here's an example of binding a point selection parameter to a radio button widget, and using the selection to filter the input data to the scatter plot.

In [ ]:
options = ['Europe', 'Japan', 'USA']
labels = [option + ' ' for option in options]

input_radio = alt.binding_radio(
    options=options + [None],
    labels=labels + ['All'],
    name='Region: '
)
selection = alt.selection_point(
    fields=['Origin'],
    value=None,
    bind=input_radio,
)

alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower:Q'),
    alt.Y('Miles_per_Gallon:Q'),
    alt.Color('Origin:N').scale(domain=options),
).add_params(
    selection
).transform_filter(
    selection
)

#### Binding point selections to click/hover
A point selection may also be bound to click or hover interactions on a mark. Here is an example that builds a point selection and uses it to control the size and color of the point that is clicked on. In addition, the `order` encoding channel is used to raise the selected point above the unselected points.

In [ ]:
selection = alt.selection_point()

alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    size=alt.condition(selection, alt.value(300), alt.value(60)),
    order=alt.condition(selection, alt.value(1), alt.value(0)),
    color=alt.condition(
        selection,
        alt.Color('Origin:N').scale(domain=['Europe', 'Japan', 'USA']),
        alt.value("lightgrey")
    )
).add_params(
    selection
)

Here are some additional options you can use with `alt.selection_point()`

```python
selection = alt.selection_point(
    on="pointerover",  # Hover instead of click
    empty=False,       # Start with all rows unselected instead of selected
    nearest=True,      # Activate nearest point to the cursor
)
```

See [Vega-Altair Selection Documentation](https://altair-viz.github.io/user_guide/generated/api/altair.selection_point.html) for the full list of configuration options.

#### Fields and Encodings Arguments
When creating point selections using `alt.selection_point`, the `fields` and `encodings` optional arguments may be used to specify how selections are made and what data they correspond to.


##### Fields Argument
The `fields` argument specifies which data columns will be used to build the selection predicate. This means that when a selection is made, only the data points that match the specified fields of the selected point will be considered selected.

To see how this works we'll set `fields` to `Origin`, which will cause all points that match the origin of the clicked point to be selected.

In [ ]:
selection = alt.selection_point(fields=["Origin"])

alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    size=alt.condition(selection, alt.value(300), alt.value(60)),
    order=alt.condition(selection, alt.value(1), alt.value(0)),
    color=alt.condition(
        selection,
        alt.Color('Origin:N').scale(domain=['Europe', 'Japan', 'USA']),
        alt.value("lightgrey")
    )
).add_params(
    selection
)

##### Encodings Argument
As an alternative to specifying the data fields to consider in the selection, the `encodings` argument allows you to specify the visual encodings that should be used for the selection. This is useful for creating interactions based on how data is represented visually, such as by color, size, or position.

Here's an example of selecting cars that match the color and size of the clicked point.


In [ ]:
selection = alt.selection_point(encodings=["size", "color"])

alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    alt.Size('Cylinders:O'),
    order=alt.condition(selection, alt.value(1), alt.value(0)),
    color=alt.condition(
        selection,
        alt.Color('Origin:N').scale(domain=['Europe', 'Japan', 'USA']),
        alt.value("lightgrey")
    )
).add_params(
    selection
)

#### Binding point selections to legend interactions
Point selection parameters may also be bound to legends, which then allows users to drive selections by clicking on legend entries. This is a powerful way to highlight or filter data points based on categorical variables represented in the legend.

A selection is bound to a legend by setting the `bind` argument to the string `'legend'`, and setting the `encodings` argument to a single element list containing the legend's encoding (e.g. `color`, `size`, etc.).

In [ ]:
# Create a point selection bound to legend
selection = alt.selection_point(
    encodings=['color'],
    bind='legend'
)

# Create chart with point selection bound to legend
chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    order=alt.condition(selection, alt.value(1), alt.value(0)),
    color=alt.condition(selection, 'Origin:N', alt.value('lightgray'))
).add_params(
    selection
)

chart

### Interval selections
Interval selection parameters are created using `alt.selection_interval()`, and they represent predicates of the form `{lower} <= {Column} < {upper}`. When an interval selection parameter is added to a chart with `add_params`, Vega-Altair adds a rectangle mark to show the selected region. By default, this rectangle mark is draggable and may be resized using scroll zoom.

Here is an example that uses an interval selection with a `condition` to control the mark's color encoding.

In [ ]:
# Define interval selection with initial values
selection = alt.selection_interval(value={'x': [50, 150], 'y': [10, 30]})

# Create chart with point selection bound to legend
chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin:N', alt.value('lightgray'))
).add_params(
    selection
)

chart

The initial interval selection `value` is not required. If not provided, nothing will be selected in the initial chart. Additionally, the `encodings` argument may be used to customize which dimensions may be selected. For example, setting `encodings` to `["x"]` will result in a horizontal box selection

In [ ]:
# Define interval selection with initial values
selection = alt.selection_interval(encodings=["x"])

# Create chart with point selection bound to legend
chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin:N', alt.value('lightgray'))
).add_params(
    selection
)

chart

### Multi-view selections
Selections are particularly useful in multi-view charts, where a selection on one view may be used to filter or highlight marks in other views.

Here we concatenate a new scatter plot, of Acceleration vs Miles per Gallon, to the right of the scatter plot above. Because the two charts are concatenated, we can use the same selection parameter in an `alt.condition` to control the color of the new scatter plot as well.

In [ ]:
selection = alt.selection_interval(encodings=['x'])

left_chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin', alt.value('lightgray'))
).add_params(
    selection
)

right_chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Acceleration'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin', alt.value('lightgray'))
)

left_chart | right_chart

You may notice that currently it's only possible to make interval selections on the left chart. This is because we've only added the selection parameter to the left chart using `add_params`. The chart that the selection is added to with `add_params` is the chart that will be configured to drive the selection. As this example demonstrates, a selection may be used in `alt.condition` (or in `transform_filter()`) in any chart in the layout whether or not the selection has been added to the chart with `add_params`.

We can update this example to support selection in both subplot by adding the selection parameter with `add_params` to the right chart as well.

In [ ]:
selection = alt.selection_interval(encodings=['x'])

left_chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin', alt.value('lightgray'))
).add_params(
    selection
)

right_chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Acceleration'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin', alt.value('lightgray'))
).add_params(
    selection
)

left_chart | right_chart

Interval selections may also be bound to scales, in which case pan and zoom are activated (rather than box selection) and the selection parameter's predicate is driven by the chart's active viewport.

Here's an example that places two identical scatter plots side by side. A selection parameter with `bind="scales"` is added to the left chart, which
 activates pan and zoom support on this chart. The selection parameter is used inside an `alt.condition` in the right chart to control the color of points. As the user pans and zooms using the left chart, the right chart highlights the points that are in the left chart's viewport.

In [ ]:
selection = alt.selection_interval(bind="scales")

left_chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    alt.Color('Origin'),
).add_params(
    selection
)

right_chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin', alt.value('lightgray'))
)

(left_chart | right_chart).properties(bounds="flush", spacing=60)

> **Note:** The `.interactive()` method described earlier is actually just a shortcut for creating an interval selection parameter and adding it to the chart!

Here's another mutli-view example that uses an interval selection in the top scatter plot to filter the inputs to the bar chart below.

In [ ]:
selection = alt.selection_interval()

scatter = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin', alt.value('lightgray'))
).add_params(
    selection
)

bars = alt.Chart(cars).mark_bar().encode(
    alt.X('count(Origin)').scale(domain=[0,260]),
    alt.Y('Origin').scale(domain=["Europe", "Japan", "USA"]),
    alt.Color('Origin'),
).transform_filter(
    selection
)

scatter & bars

## Jupyter Widget Integration
A powerful feature of Vega-Altair's support for interactivity is that it is implemented entirely in JavaScript, so these interactions work without a Python kernel (e.g. in the Vega Editor).  Sometimes, however, it's really useful to have access to a chart's interactive state in Python in order to use selections to drive other Python logic.

This is possible using [`JupyterChart`](https://altair-viz.github.io/user_guide/jupyter_chart.html), which is built on top of [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/latest/) using [AnyWidget](https://anywidget.dev/).


Here is an example of accessing an interval selection from Python. Note that it's important to name the selection so that it's easy to look up in the `jchart.selections` object.

In [ ]:
brush = alt.selection_interval(name="brush")

chart = alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
).add_params(brush)

jchart = alt.JupyterChart(chart)
jchart

In [ ]:
jchart.selections.brush.value


Here's a more complex example that uses the selection to filter a pandas DataFrame, then display that filtered pandas DataFrame below that chart in a separate widget.

In [ ]:
from ipywidgets import HTML, VBox

cars = data.cars()
brush = alt.selection_interval(name="brush")

chart_widget = alt.JupyterChart(alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
).add_params(brush))

table_widget = HTML(value=cars.iloc[:0].to_html())

def on_select(change):
    sel = change.new.value
    if sel is None or 'Horsepower' not in sel:
        filtered = cars.iloc[:0]
    else:
        filter_query = (
            f"{sel['Horsepower'][0]} <= `Horsepower` <= {sel['Horsepower'][1]} and "
            f"{sel['Miles_per_Gallon'][0]} <= `Miles_per_Gallon` <= {sel['Miles_per_Gallon'][1]}"
        )
        filtered = cars.query(filter_query)

    table_widget.value = filtered.to_html()

chart_widget.selections.observe(on_select, ["brush"])

VBox([chart_widget, table_widget])